In [2]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html

2.2.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 25.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in links: https://data.pyg.org/whl/torch-2.2.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 19.6 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
from torch_geometric.utils import dropout
from torch_geometric.loader import DataLoader
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch_geometric.utils.convert import to_scipy_sparse_matrix
#from torch_geometric.utils.train_test_split_edges import torch_geometric
import torch_geometric
import networkx as nx
import numpy as np
from torch_geometric.nn import GCNConv
import sys
from torch_geometric.datasets import TUDataset
from scipy.sparse import csr_matrix
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
import Batched_GraphSage_Layer as batched_graphsage_layer
import Batched_DIFFPOOL_Assignment as batched_diffpool_assignment
import Batched_DIFFPOOL_Embedding as batched_diffpool_embedding
import Batched_DIFFPOOL_Layer as batched_diffpool_layer



class GlobalMeanPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return torch_geometric.nn.global_mean_pool(x, batch)
################################################################################
class IdenticalPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return x

################################################################################
class DIFFPOOL_Model(nn.Module):
    '''
        DIFFPOOL Mode
    '''
    def __init__(self, embedding_input_dim, embedding_num_block_layers, embedding_hid_dim, new_feature_size, assignment_input_dim,
                 assignment_num_block_layers, assignment_hid_dim, max_number_of_nodes, concat_neighborhood, prediction_hid_layers,
                 num_classes, Weight_Initializer, Bias, dropout_rate, normalize_graphsage, aggregation, act_fun,
                 concat_diffpools_outputs, num_pooling, pooling):

        super(DIFFPOOL_Model, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.Bias = Bias
        self.dropout_rate = dropout_rate
        self.aggregation = aggregation
        self.act_fun = act_fun
        self.Weight_Initializer = Weight_Initializer
        self.concat_diffpools_outputs = concat_diffpools_outputs
        self.num_pooling = num_pooling
        self.concat = concat_neighborhood
        self.pooling = pooling


        self.embedding_input_dim = embedding_input_dim
        self.embedding_num_block_layers = embedding_num_block_layers
        self.embedding_hid_dim = embedding_hid_dim
        self.embedding_output_dim = new_feature_size
        self.normalize_graphsage = normalize_graphsage

        self.assignment_input_dim = assignment_input_dim
        self.assignment_num_block_layers = assignment_num_block_layers
        self.assignment_hid_dim = assignment_hid_dim
        self.max_number_of_nodes = max_number_of_nodes
        self.assignment_output_dim = int(self.max_number_of_nodes * 0.25)


        prediction_input_dim_sum = 0

        ###################################################.    DiffPool Layers
        diffpool_layers = []
        for i in range(self.num_pooling):
            a_new_layer = batched_diffpool_layer.Batched_DiffPool_Layer(embedding_input_dim=self.embedding_input_dim,
                                                                                 embedding_num_block_layers=self.embedding_num_block_layers,
                                                                                 embedding_hid_dim=self.embedding_hid_dim,
                                                                                 embedding_output_dim=self.embedding_output_dim,
                                                                                 assignment_input_dim=self.assignment_input_dim,
                                                                                 assignment_num_block_layers=self.assignment_num_block_layers,
                                                                                 assignment_hid_dim=self.assignment_hid_dim,
                                                                                 assignment_output_dim=self.assignment_output_dim,
                                                                                 concat=self.concat, Weight_Initializer=self.Weight_Initializer,
                                                                                 Bias=self.Bias, dropout_rate=self.dropout_rate,
                                                                                 normalize_graphsage=self.normalize_graphsage,
                                                                                 aggregation=self.aggregation, act_fun=self.act_fun).to(self.device)
            diffpool_layers.append(a_new_layer)

            self.assignment_output_dim = int(self.assignment_output_dim * .25)
            self.embedding_input_dim = self.embedding_output_dim
            self.assignment_input_dim = self.embedding_output_dim
            prediction_input_dim_sum = prediction_input_dim_sum + self.embedding_output_dim
        self.diffpool_layers = nn.Sequential(*diffpool_layers).to(self.device)

        ###################################################.    Last Extra Embedding

        self.last_extra_embedding = batched_diffpool_embedding.Batched_DiffPool_Embedding_Layer(input_dim=self.embedding_output_dim,
                                                                                                embedding_num_block_layers=self.embedding_num_block_layers,
                                                                                                hid_dim=self.embedding_hid_dim, concat=self.concat,
                                                                                                embedded_dim=self.embedding_output_dim, Bias=self.Bias,
                                                                                                normalize_graphsage=self.normalize_graphsage, dropout=self.dropout_rate,
                                                                                                aggregation=self.aggregation).to(self.device)
        prediction_input_dim_sum = prediction_input_dim_sum + self.embedding_output_dim

        ###################################################.    Predictions
        self.prediction_input_dim = self.embedding_output_dim
        self.prediction_hid_layers = prediction_hid_layers
        self.num_classes = num_classes

        prediction_layers = []
        if len(self.prediction_hid_layers) == 0:
            if self.concat_diffpools_outputs:
                a_new_layer = nn.Linear(prediction_input_dim_sum, self.num_classes).to(self.device)
                prediction_layers.append(a_new_layer)
                self.prediction_model = nn.Sequential(*prediction_layers).to(self.device)
            else:
                a_new_layer = nn.Linear(self.prediction_input_dim, self.num_classes).to(self.device)
                prediction_layers.append(a_new_layer)
                self.prediction_model = nn.Sequential(*prediction_layers).to(self.device)
        else:
            if self.concat_diffpools_outputs:
                predict_input_dim = prediction_input_dim_sum
                for i in range(len(self.prediction_hid_layers)):
                    a_new_layer = nn.Linear(predict_input_dim, prediction_hid_layers[i]).to(self.device)
                    prediction_layers.append(a_new_layer)
                    predict_input_dim = prediction_hid_layers[i]
                a_new_layer = nn.Linear(predict_input_dim, self.num_classes).to(self.device)
                prediction_layers.append(a_new_layer)
                self.prediction_model = nn.Sequential(*prediction_layers).to(self.device)
            else:
                predict_input_dim = self.prediction_input_dim
                for i in range(len(self.prediction_hid_layers)):
                    a_new_layer = nn.Linear(predict_input_dim, prediction_hid_layers[i]).to(self.device)
                    prediction_layers.append(a_new_layer)
                    predict_input_dim = prediction_hid_layers[i]
                a_new_layer = nn.Linear(predict_input_dim, self.num_classes).to(self.device)
                prediction_layers.append(a_new_layer)
                self.prediction_model = nn.Sequential(*prediction_layers).to(self.device)

        if act_fun == 'ReLu':
            self.act_fun = F.relu
            print('ReLu is Selected.')
        elif act_fun == 'eLu':
            self.act_fun = nn.functional.elu
            print('eLu is Selected.')
        elif act_fun == 'tanh':
            self.act_fun = torch.tanh
            print('tanh is Selected.')
        self.act_fun_softmax = F.softmax

        mean = 0
        std = 0.1
        self.initialize_weights(Weight_Initializer, Bias, mean, std)

    def initialize_weights(model, Weight_Initializer, Bias, mean, std):
        # 1. Xavier Normal_.  2. Kaiming Normal_.  3. Uniform (0,0.1std)

        if Weight_Initializer == 1:                                             #.      1. Xavier Normal_.
            for i, module in enumerate(model.children()):
                #print(i, module)
                if isinstance(module, torch.nn.Sequential):
                    for j, module_sub in enumerate(module):
                        #print("j: ",j,module_sub)
                        if isinstance(module_sub, batched_diffpool_layer.Batched_DiffPool_Layer):
                            #print(module_sub)
                            for party in module_sub.children():
                                if isinstance(party, batched_diffpool_embedding.Batched_DiffPool_Embedding_Layer):
                                    for diff_embd_party in party.children():
                                        #print("diff_embd_party: ")#, diff_embd_party)
                                        if isinstance(diff_embd_party, torch.nn.ModuleList):
                                            #print("moduel list for diffpool embed")
                                            for diff_embed_party_in_modulelist in diff_embd_party.children():
                                                if isinstance(diff_embed_party_in_modulelist, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                                                    torch.nn.init.xavier_normal_(diff_embed_party_in_modulelist.learnable_weights.weight)
                                                    #torch.nn.init.zeros_(diff_embed_party_in_modulelist.learnable_weights.weight)
                                                    #print("diff_embed_party_in_modulelist.learnable_weights.weight: ",diff_embed_party_in_modulelist.learnable_weights.weight)
                                                    if Bias:
                                                        torch.nn.init.zeros_(diff_embed_party_in_modulelist.learnable_weights.bias)
                                                        #print(diff_embed_party_in_modulelist.learnable_weights.bias)
                                elif isinstance(party, batched_diffpool_assignment.Batched_DiffPool_Assignment_Layer):
                                    for diff_assign_party in party.children():
                                        #print("diff_embd_party: ")#, diff_embd_party)
                                        if isinstance(diff_assign_party, torch.nn.ModuleList):
                                            #print("moduel list for diffpool embed")
                                            for diff_assign_party_in_modulelist in diff_assign_party.children():
                                                if isinstance(diff_assign_party_in_modulelist, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                                                    torch.nn.init.xavier_normal_(diff_assign_party_in_modulelist.learnable_weights.weight)
                                                    #torch.nn.init.zeros_(diff_assign_party_in_modulelist.learnable_weights.weight)
                                                    #print("diff_assign_party_in_modulelist.learnable_weights.weight: ",diff_assign_party_in_modulelist.learnable_weights.weight)
                                                    if Bias:
                                                        torch.nn.init.zeros_(diff_assign_party_in_modulelist.learnable_weights.bias)
                                                        #print(diff_assign_party_in_modulelist.learnable_weights.bias)
                                        elif isinstance(diff_assign_party, torch.nn.Sequential):
                                            for diff_assign_party_in_sequential in diff_assign_party:
                                                if isinstance(diff_assign_party_in_sequential, torch.nn.Linear):
                                                    torch.nn.init.xavier_normal_(diff_assign_party_in_sequential.weight)
                                                    #torch.nn.init.zeros_(diff_assign_party_in_sequential.weight)
                                                    if Bias:
                                                        torch.nn.init.zeros_(diff_assign_party_in_sequential.bias)
                                                        #print(diff_assign_party_in_sequential.bias)
                        elif isinstance(module_sub, torch.nn.Linear):
                            #print("linear final prediction layers")
                            torch.nn.init.xavier_normal_(module_sub.weight)
                            if Bias:
                                torch.nn.init.zeros_(module_sub.bias)
                                #print(module_sub.bias)
                elif isinstance(module, batched_diffpool_embedding.Batched_DiffPool_Embedding_Layer):
                    for embd_party in module.children():
                        #print("embd_party: ")#, embd_party)
                        if isinstance(embd_party, torch.nn.ModuleList):
                            for embed_party_in_modulelist in embd_party.children():
                                if isinstance(embed_party_in_modulelist, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                                    torch.nn.init.xavier_normal_(embed_party_in_modulelist.learnable_weights.weight)
                                    #torch.nn.init.zeros_(embed_party_in_modulelist.learnable_weights.weight)
                                    #print("embed_party_in_modulelist.learnable_weights.weight: ",embed_party_in_modulelist.learnable_weights.weight)
                                    if Bias:
                                        torch.nn.init.zeros_(embed_party_in_modulelist.learnable_weights.bias)
                                        #print(embed_party_in_modulelist.learnable_weights.bias)

        if Weight_Initializer == 2:                                             #.      1. Kaiming Normal_.
            for i, module in enumerate(model.children()):
                #print(i, module)
                if isinstance(module, torch.nn.Sequential):
                    for j, module_sub in enumerate(module):
                        #print("j: ",j,module_sub)
                        if isinstance(module_sub, batched_diffpool_layer.Batched_DiffPool_Layer):
                            #print(module_sub)
                            for party in module_sub.children():
                                if isinstance(party, batched_diffpool_embedding.Batched_DiffPool_Embedding_Layer):
                                    for diff_embd_party in party.children():
                                        #print("diff_embd_party: ")#, diff_embd_party)
                                        if isinstance(diff_embd_party, torch.nn.ModuleList):
                                            #print("moduel list for diffpool embed")
                                            for diff_embed_party_in_modulelist in diff_embd_party.children():
                                                if isinstance(diff_embed_party_in_modulelist, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                                                    torch.nn.init.kaiming_normal_(diff_embed_party_in_modulelist.learnable_weights.weight)
                                                    #torch.nn.init.zeros_(diff_embed_party_in_modulelist.learnable_weights.weight)
                                                    #print("diff_embed_party_in_modulelist.learnable_weights.weight: ",diff_embed_party_in_modulelist.learnable_weights.weight)
                                                    if Bias:
                                                        torch.nn.init.zeros_(diff_embed_party_in_modulelist.learnable_weights.bias)
                                                        #print(diff_embed_party_in_modulelist.learnable_weights.bias)
                                elif isinstance(party, batched_diffpool_assignment.Batched_DiffPool_Assignment_Layer):
                                    for diff_assign_party in party.children():
                                        #print("diff_embd_party: ")#, diff_embd_party)
                                        if isinstance(diff_assign_party, torch.nn.ModuleList):
                                            #print("moduel list for diffpool embed")
                                            for diff_assign_party_in_modulelist in diff_assign_party.children():
                                                if isinstance(diff_assign_party_in_modulelist, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                                                    torch.nn.init.kaiming_normal_(diff_assign_party_in_modulelist.learnable_weights.weight)
                                                    #torch.nn.init.zeros_(diff_assign_party_in_modulelist.learnable_weights.weight)
                                                    #print("diff_assign_party_in_modulelist.learnable_weights.weight: ",diff_assign_party_in_modulelist.learnable_weights.weight)
                                                    if Bias:
                                                        torch.nn.init.zeros_(diff_assign_party_in_modulelist.learnable_weights.bias)
                                                        #print(diff_assign_party_in_modulelist.learnable_weights.bias)
                                        elif isinstance(diff_assign_party, torch.nn.Sequential):
                                            for diff_assign_party_in_sequential in diff_assign_party:
                                                if isinstance(diff_assign_party_in_sequential, torch.nn.Linear):
                                                    torch.nn.init.kaiming_normal_(diff_assign_party_in_sequential.weight)
                                                    #torch.nn.init.zeros_(diff_assign_party_in_sequential.weight)
                                                    if Bias:
                                                        torch.nn.init.zeros_(diff_assign_party_in_sequential.bias)
                                                        #print(diff_assign_party_in_sequential.bias)
                        elif isinstance(module_sub, torch.nn.Linear):
                            #print("linear final prediction layers")
                            torch.nn.init.kaiming_normal_(module_sub.weight)
                            if Bias:
                                torch.nn.init.zeros_(module_sub.bias)
                                #print(module_sub.bias)
                elif isinstance(module, batched_diffpool_embedding.Batched_DiffPool_Embedding_Layer):
                    for embd_party in module.children():
                        #print("embd_party: ")#, embd_party)
                        if isinstance(embd_party, torch.nn.ModuleList):
                            for embed_party_in_modulelist in embd_party.children():
                                if isinstance(embed_party_in_modulelist, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                                    torch.nn.init.kaiming_normal_(embed_party_in_modulelist.learnable_weights.weight)
                                    #torch.nn.init.zeros_(embed_party_in_modulelist.learnable_weights.weight)
                                    #print("embed_party_in_modulelist.learnable_weights.weight: ",embed_party_in_modulelist.learnable_weights.weight)
                                    if Bias:
                                        torch.nn.init.zeros_(embed_party_in_modulelist.learnable_weights.bias)
                                        #print(embed_party_in_modulelist.learnable_weights.bias)

        if Weight_Initializer == 3:                                             #.      3. Uniform (0,0.1std)
            for i, module in enumerate(model.children()):
                #print(i, module)
                if isinstance(module, torch.nn.Sequential):
                    for j, module_sub in enumerate(module):
                        #print("j: ",j,module_sub)
                        if isinstance(module_sub, batched_diffpool_layer.Batched_DiffPool_Layer):
                            #print(module_sub)
                            for party in module_sub.children():
                                if isinstance(party, batched_diffpool_embedding.Batched_DiffPool_Embedding_Layer):
                                    for diff_embd_party in party.children():
                                        #print("diff_embd_party: ")#, diff_embd_party)
                                        if isinstance(diff_embd_party, torch.nn.ModuleList):
                                            #print("moduel list for diffpool embed")
                                            for diff_embed_party_in_modulelist in diff_embd_party.children():
                                                if isinstance(diff_embed_party_in_modulelist, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                                                    torch.nn.init.normal_(diff_embed_party_in_modulelist.learnable_weights.weight, mean=mean, std=std)
                                                    #torch.nn.init.zeros_(diff_embed_party_in_modulelist.learnable_weights.weight)
                                                    #print("diff_embed_party_in_modulelist.learnable_weights.weight: ",diff_embed_party_in_modulelist.learnable_weights.weight)
                                                    if Bias:
                                                        torch.nn.init.zeros_(diff_embed_party_in_modulelist.learnable_weights.bias)
                                                        #print(diff_embed_party_in_modulelist.learnable_weights.bias)
                                elif isinstance(party, batched_diffpool_assignment.Batched_DiffPool_Assignment_Layer):
                                    for diff_assign_party in party.children():
                                        #print("diff_embd_party: ")#, diff_embd_party)
                                        if isinstance(diff_assign_party, torch.nn.ModuleList):
                                            #print("moduel list for diffpool embed")
                                            for diff_assign_party_in_modulelist in diff_assign_party.children():
                                                if isinstance(diff_assign_party_in_modulelist, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                                                    torch.nn.init.normal_(diff_assign_party_in_modulelist.learnable_weights.weight, mean=mean, std=std)
                                                    #torch.nn.init.zeros_(diff_assign_party_in_modulelist.learnable_weights.weight)
                                                    #print("diff_assign_party_in_modulelist.learnable_weights.weight: ",diff_assign_party_in_modulelist.learnable_weights.weight)
                                                    if Bias:
                                                        torch.nn.init.zeros_(diff_assign_party_in_modulelist.learnable_weights.bias)
                                                        #print(diff_assign_party_in_modulelist.learnable_weights.bias)
                                        elif isinstance(diff_assign_party, torch.nn.Sequential):
                                            for diff_assign_party_in_sequential in diff_assign_party:
                                                if isinstance(diff_assign_party_in_sequential, torch.nn.Linear):
                                                    torch.nn.init.normal_(diff_assign_party_in_sequential.weight, mean=mean, std=std)
                                                    #torch.nn.init.zeros_(diff_assign_party_in_sequential.weight)
                                                    if Bias:
                                                        torch.nn.init.zeros_(diff_assign_party_in_sequential.bias)
                                                        #print(diff_assign_party_in_sequential.bias)
                        elif isinstance(module_sub, torch.nn.Linear):
                            #print("linear final prediction layers")
                            torch.nn.init.normal_(module_sub.weight, mean=mean, std=std)
                            if Bias:
                                torch.nn.init.zeros_(module_sub.bias)
                                #print(module_sub.bias)
                elif isinstance(module, batched_diffpool_embedding.Batched_DiffPool_Embedding_Layer):
                    for embd_party in module.children():
                        #print("embd_party: ")#, embd_party)
                        if isinstance(embd_party, torch.nn.ModuleList):
                            for embed_party_in_modulelist in embd_party.children():
                                if isinstance(embed_party_in_modulelist, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                                    torch.nn.init.normal_(embed_party_in_modulelist.learnable_weights.weight, mean=mean, std=std)
                                    #torch.nn.init.zeros_(embed_party_in_modulelist.learnable_weights.weight)
                                    #print("embed_party_in_modulelist.learnable_weights.weight: ",embed_party_in_modulelist.learnable_weights.weight)
                                    if Bias:
                                        torch.nn.init.zeros_(embed_party_in_modulelist.learnable_weights.bias)
                                        #print(embed_party_in_modulelist.learnable_weights.bias)







    def computational_matricess(self, batched_graphs, edge_mask):
        if edge_mask == None:
            joint_tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense()) + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense())))
        else:
            joint_tilda_adjacency_matrix = torch.tensor(csr_matrix((np.array(edge_mask), (np.array(batched_graphs.edge_index[0]), np.array(batched_graphs.edge_index[1])))).todense())

        joint_tilda_adjacency_matrix = joint_tilda_adjacency_matrix.type(torch.float32)


        if batched_graphs.batch is not None:
            graph_sizes = [len(batched_graphs[i].x) for i in range(len(batched_graphs))]
            batch_size = batched_graphs.num_graphs
        else:
            graph_sizes = [len(batched_graphs.x)]
            batch_size = 1
        max_number_of_nodes_in_batch_of_graphs = max(graph_sizes)
        #print("max_number_of_nodes_in_batch_of_graphs: ", max_number_of_nodes_in_batch_of_graphs)

        new_number_of_nodes = int(joint_tilda_adjacency_matrix.size()[0] / batch_size)

        adjacency_list = []
        feature_list = []
        start = 0
        for i in range(batch_size):
            end = start + graph_sizes[i]
            un_padded_adj = joint_tilda_adjacency_matrix[start:end, start:end]
            adj_off_set = max_number_of_nodes_in_batch_of_graphs - un_padded_adj.size()[0]
            if un_padded_adj.size()[0] <= max_number_of_nodes_in_batch_of_graphs:
                un_padded_adj = F.pad(un_padded_adj, (0, adj_off_set, 0, adj_off_set), mode='constant', value=0)
                un_padded_adj = un_padded_adj.type(torch.float32)
                num_nodes = max_number_of_nodes_in_batch_of_graphs
            un_padded_adj = un_padded_adj.type(torch.float32)
            adjacency_list.append(un_padded_adj)
            un_padded_feat = batched_graphs.x[start:end, :]
            node_feat_off_set = max_number_of_nodes_in_batch_of_graphs - graph_sizes[i]
            un_padded_feat = F.pad(un_padded_feat, (0, 0, 0, node_feat_off_set), mode='constant', value=0)
            un_padded_feat = un_padded_feat.type(torch.float32)
            un_padded_feat.require_grad = True
            feature_list.append(un_padded_feat)
            start = end

        adjacency_list = list(map(lambda x: torch.unsqueeze(x, 0), adjacency_list))
        feature_list = list(map(lambda x: torch.unsqueeze(x, 0), feature_list))

        new_adjacecny = torch.cat(adjacency_list, dim=0)
        new_features = torch.cat(feature_list, dim=0)

        return new_adjacecny, new_features

    def forward(self, batched_graphs, edge_mask):
        adjacecny, features = self.computational_matricess(batched_graphs, edge_mask)
        adjacecny = adjacecny.to(self.device)
        features = features.to(self.device)
        concatination_list_of_poolings = []

        for i in range(self.num_pooling):
            embedding_output, assignment_output = self.diffpool_layers[i](features, adjacecny)
            #features = torch.matmul(torch.transpose(assignment_output, 1, 2), embedding_output)
            embedding_output = embedding_output.to(self.device)
            assignment_output = assignment_output.to(self.device)
            features = torch.bmm(torch.transpose(assignment_output, 1, 2), embedding_output)
            adjacecny = torch.transpose(assignment_output, 1, 2) @ adjacecny @ assignment_output


            if self.pooling == "max":
                embedding_output_pooled, _ = torch.max(embedding_output, dim=1)
            elif self.pooling == "mean":
                embedding_output_pooled = torch.mean(embedding_output, dim=1)
            elif self.pooling == "sum":
                embedding_output_pooled = torch.sum(embedding_output, dim=1)
            concatination_list_of_poolings.append(embedding_output_pooled)


        extra_embed = self.last_extra_embedding(features, adjacecny)
        if self.pooling == "max":
            extra_embed_pooled, _ = torch.max(extra_embed, dim=1)
        elif self.pooling == "mean":
            extra_embed_pooled = torch.mean(extra_embed, dim=1)
        elif self.pooling == "sum":
            extra_embed_pooled = torch.sum(extra_embed, dim=1)
        concatination_list_of_poolings.append(extra_embed_pooled)

        if self.concat_diffpools_outputs:
            output = torch.cat(concatination_list_of_poolings, dim=1)
        else:
            output = extra_embed_pooled

        prediction_output = output
        for i in range(len(self.prediction_hid_layers)):

            prediction_output = self.act_fun(self.prediction_model[i](prediction_output))
        prediction_output = self.prediction_model[-1](prediction_output)
        prediction_output_soft = F.softmax(prediction_output, dim=-1)



        return concatination_list_of_poolings, prediction_output, prediction_output_soft







diffpool_model_example = DIFFPOOL_Model(embedding_input_dim=7, embedding_num_block_layers=1, embedding_hid_dim=64, new_feature_size=20,
                                        assignment_input_dim=7, assignment_num_block_layers=1, assignment_hid_dim=64, max_number_of_nodes=128,
                                        prediction_hid_layers=[7], concat_neighborhood=False, num_classes=2, Weight_Initializer=3,
                                        Bias=True, dropout_rate=0, normalize_graphsage=True, aggregation="mean",
                                        act_fun="ReLu", concat_diffpools_outputs=False, num_pooling=1, pooling="sum")


dataset = TUDataset(root='data/TUDataset', name='MUTAG')
batch_size = 10
batched_dataset = DataLoader(dataset, batch_size=batch_size, shuffle=False)
GlobalMeanPool_exp = GlobalMeanPool()
for batched_graph in batched_dataset:
    #x, edge_index, batch, y = batched_graph.x, batched_graph.edge_index, batched_graph.batch, batched_graph.y
    concatination_list_of_poolings, prediction_output, prediction_output_soft = diffpool_model_example(batched_graph, None)
    print("Final Output: ", prediction_output_soft)



ReLu is Selected.
Final Output:  tensor([[0.4570, 0.5430],
        [0.4682, 0.5318],
        [0.4686, 0.5314],
        [0.4543, 0.5457],
        [0.4720, 0.5280],
        [0.4916, 0.5084],
        [0.4586, 0.5414],
        [0.4708, 0.5292],
        [0.4683, 0.5317],
        [0.4570, 0.5430]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Final Output:  tensor([[0.4570, 0.5430],
        [0.4573, 0.5427],
        [0.4739, 0.5261],
        [0.4693, 0.5307],
        [0.4543, 0.5457],
        [0.4740, 0.5260],
        [0.4671, 0.5329],
        [0.4731, 0.5269],
        [0.4685, 0.5315],
        [0.4601, 0.5399]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Final Output:  tensor([[0.4839, 0.5161],
        [0.4719, 0.5281],
        [0.4505, 0.5495],
        [0.4478, 0.5522],
        [0.4730, 0.5270],
        [0.4940, 0.5060],
        [0.4505, 0.5495],
        [0.4583, 0.5417],
        [0.4893, 0.5107],
        [0.4505, 0.5495]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Final Output:  tens

In [ ]:
print(np.zeros(1,dtype=np.float32))

[0.]
